In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[4],3b
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[6]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.000410                    1  0.000335
0.000750        2  0.000465                    2  0.000514
0.001052        3  0.000588                    3  0.000389
0.001476        4  0.000733                    4  0.000477
0.002070        5  0.000899                    5  0.000581
0.002904        6  0.001089                    6  0.000699
0.004074        7  0.001306                    7  0.000846
0.005714        8  0.001554                    8  0.001021
0.008015        9  0.001836                    9  0.001219
0.011243       10  0.002155                   10  0.001440
0.015771       11  0.002515                   11  0.001687
0.022122       12  0.002922                   12  0.001965
0.031031       13  0.003377                   13  0.002274
0.043528       14  0.003821                   14  0.002579
0.061057       15  0.004196                   15  0.002840
0.085645       16  0.004539                   16  0.003081
0.120136       17  0.004900                   17  0.003334
0.168516       18  0.005277                   18  0.003596
0.236378       19  0.005675                   19  0.003868
0.331549       20  0.006090                   20  0.004153
0.465100       21  0.006474                   21  0.004437
0.652400       22  0.006096                   22  0.004242
0.915100       23  0.004839                   23  0.003424
1.283650       24  0.003592                   24  0.002582
1.800600       25  0.002617                   25  0.001911
2.525700       26  0.001864                   26  0.001387
3.542800       27  0.001294                   27  0.000983
4.969550       28  0.000916                   28  0.000711
6.970850       29  0.000703                   29  0.000553
9.778100       30  0.000563                   30  0.000446
13.715850      31  0.000454                   31  0.000360
19.239350      32  0.000374                   32  0.000295
26.987250      33  0.000353                   33  0.000275
37.855300      34  0.000389                   34  0.000297
53.100050      35  0.000436                   35  0.000337
73.887500      36  0.000480                   36  0.000380
97.662500      37  0.000509                   37  0.000410
121.437500     38  0.000514                   38  0.000418
145.212500     39  0.000688                   39  0.000561
168.987500     40  0.000862                   40  0.000711
192.762500     41  0.001042                   41  0.000875
216.537500     42  0.001146                   42  0.000989
240.312500     43  0.001224                   43  0.001089
264.087500     44  0.001299                   44  0.001189
287.862500     45  0.001387                   45  0.001308
311.637500     46  0.001701                   46  0.001635
335.412500     47  0.002496                   47  0.002389
359.187500     48  0.003752                   48  0.003583
382.962500     49  0.006718                   49  0.006468
406.737500     50  0.011882                   50  0.011573
430.512500     51  0.018756                   51  0.018380
454.287500     52  0.027066                   52  0.026598
478.062500     53  0.036376                   53  0.035790
501.837500     54  0.046272                   54  0.045680
525.612500     55  0.056390                   55  0.055743
549.387500     56  0.066418                   56  0.065802
573.162500     57  0.076172                   57  0.075120
596.937500     58  0.085436                   58  0.083178
620.712500     59  0.094052                   59  0.090023
644.487500     60  0.101851                   60  0.096071
668.262500     61  0.108711                   61  0.101748
692.037500     62  0.114576                   62  0.107216
715.812500     63  0.119443                   63  0.112228
739.587500     64  0.123401                   64  0.116519
763.362500     65  0.126636                   65  0.119971
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -26.440622  0.000000e+00 -26.440622 -26.447360   
0.000624    2     -26.440622  5.335529e-08 -26.440621 -26.447360   
0.000876    3     -26.440622  7.618478e-08 -26.440621 -26.447360   
0.001229    4     -26.440622  1.123454e-07 -26.440621 -26.447360   
0.001723    5     -26.440622  1.696780e-07 -26.440621 -26.447360   
0.002417    6     -26.440622  2.604617e-07 -26.440621 -26.447360   
0.003391    7     -26.440622  4.040599e-07 -26.440621 -26.447360   
0.004757    8     -26.440622  6.309534e-07 -26.440621 -26.447360   
0.006672    9     -26.440622  9.890462e-07 -26.440621 -26.447360   
0.009359    10    -26.440622  1.553599e-06 -26.440620 -26.447360   
0.013128    11    -26.440622  2.442678e-06 -26.440619 -26.447360   
0.018415    12    -26.440621  3.841356e-06 -26.440617 -26.447360   
0.025830    13    -26.440621  6.039353e-06 -26.440615 -26.447360   
0.036232    14    -26.440620  9.487809e-06 -26.440611 -26.447360   
0.050823    15    -26.440619  1.481563e-05 -26.440604 -26.447359   
0.071291    16    -26.440617  2.286054e-05 -26.440594 -26.447357   
0.100000    17    -26.440613  3.486909e-05 -26.440579 -26.447355   
0.140271    18    -26.440608  5.275653e-05 -26.440555 -26.447352   
0.196760    19    -26.440599  7.933950e-05 -26.440520 -26.447346   
0.275997    20    -26.440585  1.187710e-04 -26.440467 -26.447337   
0.387100    21    -26.440564  1.770506e-04 -26.440387 -26.447323   
0.543100    22    -26.440529  2.624633e-04 -26.440267 -26.447302   
0.761700    23    -26.440484  3.749020e-04 -26.440108 -26.447274   
1.068500    24    -26.440436  5.031053e-04 -26.439933 -26.447246   
1.498800    25    -26.440392  6.416976e-04 -26.439750 -26.447223   
2.102400    26    -26.440353  7.902766e-04 -26.439563 -26.447207   
2.949000    27    -26.440324  9.484222e-04 -26.439376 -26.447201   
4.136600    28    -26.440310  1.116278e-03 -26.439194 -26.447212   
5.802500    29    -26.440314  1.301003e-03 -26.439013 -26.447242   
8.139200    30    -26.440338  1.519651e-03 -26.438819 -26.447295   
11.417000   31    -26.440390  1.790136e-03 -26.438600 -26.447380   
16.014700   32    -26.440484  2.131251e-03 -26.438353 -26.447510   
22.464000   33    -26.440640  2.572858e-03 -26.438067 -26.447704   
31.510500   34    -26.440878  3.189985e-03 -26.437688 -26.447985   
44.200100   35    -26.441221  4.117952e-03 -26.437104 -26.448376   
62.000000   36    -26.441709  5.525099e-03 -26.436184 -26.448936   
85.775000   37    -26.442373  7.539811e-03 -26.434832 -26.449705   
109.550000  38    -26.443056  9.654763e-03 -26.433401 -26.450495   
133.325000  39    -26.443779  1.182748e-02 -26.431951 -26.451317   
157.100000  40    -26.444845  1.483006e-02 -26.430014 -26.452495   
180.875000  41    -26.446331  1.874326e-02 -26.427588 -26.454097   
204.650000  42    -26.448351  2.369836e-02 -26.424653 -26.456230   
228.425000  43    -26.450867  2.944272e-02 -26.421424 -26.458861   
252.200000  44    -26.453933  3.595363e-02 -26.417979 -26.462038   
275.975000  45    -26.457681  4.335972e-02 -26.414322 -26.465877   
299.750000  46    -26.462260  5.184512e-02 -26.410414 -26.470497   
323.525000  47    -26.467724  6.210166e-02 -26.405623 -26.475952   
347.300000  48    -26.474025  7.543249e-02 -26.398593 -26.482193   
371.075000  49    -26.481084  9.305808e-02 -26.388027 -26.489188   
394.850000  50    -26.490895  1.217879e-01 -26.369107 -26.498960   
418.625000  51    -26.505155  1.695108e-01 -26.335645 -26.513199   
442.400000  52    -26.524625  2.418041e-01 -26.282821 -26.532594   
466.175000  53    -26.549831  3.432375e-01 -26.206593 -26.557681   
489.950000  54    -26.580960  4.768133e-01 -26.104147 -26.588641   
513.725000  55    -26.617853  6.440219e-01 -25.973831 -26.625328   
537.500000  56    -26.659993  8.449746e-01 -25.815018 -26.667021   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')